In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import altair as alt

In [2]:
data = pd.read_csv('final_data.csv')

f:\Lampros\Analysis of Delegates Voting Patterns in Arbitrum Proposals\Analysis\data_analysis\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.shape

(3719949, 22)

In [4]:
data.columns

Index(['Vote_ID', 'Voter', 'Voting_Power', 'Vote_Creation_Date',
       'Voting_Reason', 'Vote_Year', 'Vote_Month', 'Vote_Day', 'Vote_Time',
       'Proposal_ID', 'Proposal_Title', 'Proposal_Body', 'Proposal_Type',
       'Proposal_Creation_Date', 'Proposal_Start_Date', 'Proposal_End_Date',
       'Proposal_Year', 'Proposal_Month', 'Proposal_Day', 'Proposal_Time',
       'Proposal_Choices', 'Voting_Choice'],
      dtype='object')

In [5]:
# Convert 'Vote_Creation_Date' column to datetime format
data['Vote_Creation_Date'] = pd.to_datetime(data['Vote_Creation_Date'])

# Extracting timestamp (date and time) from Vote_Creation_Date
data['Timestamp'] = data['Vote_Creation_Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Finding the timestamp with maximum occurrences
max_timestamp_votes = data['Timestamp'].value_counts().idxmax()

print("Timestamp with Maximum Votes Received:", max_timestamp_votes)

Timestamp with Maximum Votes Received: 2023-10-09 05:53:10


In [32]:
# Count occurrences of each timestamp
timestamp_counts = data['Timestamp'].value_counts()

# Get the top 10 timestamps with maximum votes
timestamps = timestamp_counts.head(10)

# Convert the Series to a DataFrame
timestamps = timestamps.reset_index()

# Rename the columns
timestamps.columns = ['Timestamp', 'Vote Count']

# Display the DataFrame
print("DataFrame of Top 10 Timestamps with Maximum Votes:")
timestamps

DataFrame of Top 10 Timestamps with Maximum Votes:


,Timestamp,Vote Count
0,2023-10-09 05:53:10,118
1,2023-10-09 05:33:51,118
2,2023-10-09 07:56:21,113
3,2023-10-09 07:03:46,108
4,2023-10-08 19:07:00,107
5,2023-10-08 22:46:54,107
6,2023-10-08 17:37:10,106
7,2023-10-07 11:46:27,106
8,2023-10-08 19:09:28,106
9,2023-10-08 19:01:16,106


In [34]:
# Sort the DataFrame by timestamp
timestamps_sorted = timestamps.sort_values(by='Timestamp')

# Display the sorted DataFrame
print("Sorted DataFrame of Top 10 Timestamps with Maximum Votes:")
timestamps_sorted

Sorted DataFrame of Top 10 Timestamps with Maximum Votes:


,Timestamp,Vote Count
7,2023-10-07 11:46:27,106
6,2023-10-08 17:37:10,106
9,2023-10-08 19:01:16,106
4,2023-10-08 19:07:00,107
8,2023-10-08 19:09:28,106
5,2023-10-08 22:46:54,107
1,2023-10-09 05:33:51,118
0,2023-10-09 05:53:10,118
3,2023-10-09 07:03:46,108
2,2023-10-09 07:56:21,113


In [35]:
# Create a bar chart
fig = px.bar(timestamps_sorted, x='Timestamp', y='Vote Count', 
             labels={'Timestamp': 'Timestamp', 'Vote Count': 'Vote Count'},
             title='Top 10 Timestamps with Maximum Votes')

# Show the chart
fig.show()

In [39]:
# Create a line chart
fig = px.line(timestamps_sorted, x='Timestamp', y='Vote Count', 
              labels={'Timestamp': 'Timestamp', 'Vote Count': 'Vote Count'},
              title='Top 10 Timestamps with Maximum Votes')

# Update the line chart to include markers
fig.update_traces(mode='markers+lines')

# Show the chart
fig.show()
fig.write_html(r'Graphs\top-10-timestamp.html')

In [10]:
# Filter the DataFrame for rows with the maximum timestamp
max_timestamp_votes_df = data[data['Timestamp'] == max_timestamp_votes]

# Count the number of votes on the maximum timestamp
num_votes_max_timestamp = len(max_timestamp_votes_df)

print("Number of Votes on Timestamp with Maximum Occurrences:", num_votes_max_timestamp)

Number of Votes on Timestamp with Maximum Occurrences: 118


In [11]:
# Filter the DataFrame for rows with the maximum timestamp
max_timestamp_votes_df = data[data['Timestamp'] == max_timestamp_votes]

# Find the proposal title with the maximum votes
max_votes_proposal_title = max_timestamp_votes_df.loc[max_timestamp_votes_df['Voting_Power'].idxmax(), 'Proposal_Title']

print("Proposal Title with Maximum Votes on the Timestamp with Maximum Occurrences:", max_votes_proposal_title)

Proposal Title with Maximum Votes on the Timestamp with Maximum Occurrences: StakeDAO STIP Proposal - Round 1


In [12]:
# Grouping the data by 'Proposal_Title' and calculating the sum of 'Voting_Power'
proposal_voting_power = data.groupby('Proposal_Title')['Voting_Power'].sum().reset_index()

# Finding the proposal with the highest sum of voting power
proposal_highest_voting_power = proposal_voting_power.loc[proposal_voting_power['Voting_Power'].idxmax()]

print("Proposal with the Highest Voting Power:")
print(proposal_highest_voting_power)

Proposal with the Highest Voting Power:
Proposal_Title    The Arbitrum Coalition
Voting_Power            240473800.221014
Name: 117, dtype: object


In [13]:
# Group the data by 'proposal_id' and 'proposal_title' and count the votes for each proposal
proposal_vote_counts = data.groupby(['Proposal_ID', 'Proposal_Title']).size().reset_index(name='total_votes')

proposal_vote_counts

,Proposal_ID,Proposal_Title,total_votes
0,0x029f9208ec4560c5c4868bcf1a5633ffb15aa69f5000...,SpartaDEX STIP Proposal - Round 1,18001
1,0x03e61c8225487f2ff6305c91ce37af31853e6c09f557...,TIDE STIP Proposal - Round 1,15873
2,0x070a960cf0d8824badc07f0478040a16e92959c71b6e...,Curve STIP Proposal - Round 1,21353
3,0x07a26cd6b78a41745aab04190f22e97fdf9432f56465...,Proposal: [Non-Constitutional] Funding for Int...,25843
4,0x0908edd135f544cdf93dd628bf9813b0f3af517c6d87...,RabbitHole STIP Proposal - Round 1,18303
...,...,...,...
138,0xfa78979a7afa0b0df5c885ebf3a0d46c3676152c6c95...,Domain Allocator Election for the New Protocol...,49436
139,0xfbf8cd22584d70171a95dca6b1dbfbd1a9d1ba3ee1d3...,GMD Ecosystem STIP Proposal - Round 1,25902
140,0xfc76965b400c55344ca9aa3134c17492b0c458567196...,Premia STIP Proposal - Round 1,17246
141,0xfd3551e2a0effc5d900e522b79300f68c351ec930cb0...,Proposal: Security Council Elections Proposed ...,53506


In [14]:
# Sort the DataFrame by 'total_votes' column in descending order
top_30_proposals = proposal_vote_counts.sort_values(by='total_votes', ascending=False).head(30)

# Create the bar chart using Plotly
fig = px.bar(top_30_proposals, x='Proposal_Title', y='total_votes',
             labels={'Proposal_Title': 'Proposals', 'total_votes': 'Total Votes'},
             title='Top 30 Proposals by Total Votes',
             color='total_votes',
             color_continuous_scale='Viridis')

# Remove x-ticks
fig.update_xaxes(showticklabels=False)

# Set x-axis label
fig.update_layout(xaxis=dict(title='Proposals'))

# Show the interactive plot
fig.show()
fig.write_html(r'Graphs\top-30-votes.html')

In [15]:
# Sort the data based on total_votes in descending order
sorted_proposals = proposal_vote_counts.sort_values(by='total_votes', ascending=False)

# Select the proposal with the maximum votes
proposal_with_max_votes = sorted_proposals.iloc[0]

print("Proposal with Maximum Votes:")
proposal_with_max_votes

Proposal with Maximum Votes:


Proposal_ID       0x6fc671517d63e69232fafd581e9461c50150008aa61f...
Proposal_Title    Arbitrum as official sponsor of Ethereum Mexic...
total_votes                                                   53602
Name: 65, dtype: object

In [16]:
proposal_vote_counts[proposal_vote_counts['total_votes'] == 53602]['Proposal_Title']

65    Arbitrum as official sponsor of Ethereum Mexic...
Name: Proposal_Title, dtype: object

In [17]:
# Count unique values and their counts for the 'voter' column
voter_counts = data['Voter'].value_counts()

# Create a DataFrame from the 'voter_counts' Series
voter_counts_df = pd.DataFrame({'Voter': voter_counts.index, 'votes_count': voter_counts.values})

voter_counts_df

,Voter,votes_count
0,0x07BF41ade9d3499c5e5e7E6c16aEd5D76eEaB9ad,143
1,0x62bcFD62DA9dbc7382C203B9759f62bc1356f5d4,143
2,0x2Eeaa7534aA31545E98319ed72827dA53264775f,143
3,0xCf21C3bBCc34aFa5D474cfF53752F8eE8395bf9b,143
4,0x2E3C66954C617Ee0C1977DE94b711107d5126721,143
...,...,...
152972,0xC82d2Cf8c1b1AFDF2a35662090f71EbeF492601b,1
152973,0x3A299dc60965a03252caEd8De404d955eA641F84,1
152974,0x55b61DC81A1BaEA734B075934a509D3eBc1c9576,1
152975,0xa1D7ADE303fAE691A1aa11E85D7fc77F496ed683,1


In [18]:
len(voter_counts_df[voter_counts_df['votes_count'] == 143])

177

In [19]:
voters = voter_counts_df[voter_counts_df['votes_count'] == 143]['Voter'].to_list()

In [20]:
voters

['0x07BF41ade9d3499c5e5e7E6c16aEd5D76eEaB9ad',
 '0x62bcFD62DA9dbc7382C203B9759f62bc1356f5d4',
 '0x2Eeaa7534aA31545E98319ed72827dA53264775f',
 '0xCf21C3bBCc34aFa5D474cfF53752F8eE8395bf9b',
 '0x2E3C66954C617Ee0C1977DE94b711107d5126721',
 '0xD5d74BffEC4b1C945094383844Aea8Bd93073c4D',
 '0x512010DF9b2BB933CeD5d49e8eB843718847ACC1',
 '0xe44782AFE398800d7190796018C4b865E32fF806',
 '0xC2322C65333CcDDcf07FA46E91d6A4b013412505',
 '0xc077c537727b07a6521332dA75Fa8966192f425d',
 '0xC55D9828B29f7b1c047cdF45CafEe268aC6601c6',
 '0x7D3b6F0fae542B06a071C8d7ad7B06AB8ad7F16a',
 '0xA22628aF5731EE4ED03817606E9412F842BE60F5',
 '0x7ccFFE5EBE29509147e5f72c772f47a98C1907B7',
 '0xfbD0110b7F5F01D42C89Bce9aB9b1918949587ea',
 '0xAcE58f7084cBC3d2e1C178ce4B038Bb84053352b',
 '0x309156fCB6d32e6A117E578553DF76b326B62e28',
 '0x4bCA7C35c62d01044E46B0a87fAf014Ab74D5f54',
 '0xDe96e75c7160d70a447a72AFdb75DDfA1455c808',
 '0x2A5A9Da096f9A246fEf6F838132460f5eE8CBe6A',
 '0xABE3496FA08355Cf4A3CF4772053615396a8E4e9',
 '0xe614CDfE6

In [21]:
# Generate HTML table
html_table = "<table border='1'><tr><th>Voter</th></tr>"
for voter in voters:
    html_table += f"<tr><td>{voter}</td></tr>"
html_table += "</table>"

# Write HTML to file
with open(r'voters_table.html', 'w') as f:
    f.write(html_table)

In [22]:
len(voter_counts_df[voter_counts_df['votes_count'] == 1])

13520

In [23]:
data.columns

Index(['Vote_ID', 'Voter', 'Voting_Power', 'Vote_Creation_Date',
       'Voting_Reason', 'Vote_Year', 'Vote_Month', 'Vote_Day', 'Vote_Time',
       'Proposal_ID', 'Proposal_Title', 'Proposal_Body', 'Proposal_Type',
       'Proposal_Creation_Date', 'Proposal_Start_Date', 'Proposal_End_Date',
       'Proposal_Year', 'Proposal_Month', 'Proposal_Day', 'Proposal_Time',
       'Proposal_Choices', 'Voting_Choice', 'Timestamp'],
      dtype='object')

In [24]:
# Assuming your DataFrame is named 'merged_df'
proposal_counts = data.groupby(['Proposal_Year', 'Proposal_Month'])['Proposal_ID'].nunique().reset_index()
proposal_counts.columns = ['Proposal_Year', 'Proposal_Month', 'proposal_count']

# Calculate the percentage
total_proposals = proposal_counts['proposal_count'].sum()
proposal_counts['percentage'] = round(((proposal_counts['proposal_count'] / total_proposals) * 100), 2)

# Display the result
print(proposal_counts)

    Proposal_Year  Proposal_Month  proposal_count  percentage
0            2023               3               1        0.70
1            2023               4               3        2.10
2            2023               6               2        1.40
3            2023               7               3        2.10
4            2023               8               4        2.80
5            2023               9               8        5.59
6            2023              10             100       69.93
7            2023              11               7        4.90
8            2023              12               4        2.80
9            2024               1               7        4.90
10           2024               2               4        2.80


In [25]:
# Map month numbers to month names
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# Create figure
fig = go.Figure()

# Add trace for proposal counts (line chart)
fig.add_trace(go.Scatter(x=proposal_counts.apply(lambda row: f"{month_names[row['Proposal_Month']]} {int(row['Proposal_Year'])}", axis=1),
                         y=proposal_counts['proposal_count'],
                         mode='lines+markers',
                         marker=dict(size=6),
                         line=dict(color='green'),
                         name='Proposals Count'))

# Update layout
fig.update_layout(title='Proposals Count by Year-Month',
                  xaxis=dict(title='Year-Month', tickangle=-45),
                  yaxis=dict(title='Proposals Count'))

# Display the chart
fig.show()
fig.write_html(r'Graphs\line-proposals-month.html')

In [26]:
# Group the data by Proposal_Type and count the unique Proposal_IDs
unique_proposals_count = data.groupby('Proposal_Type')['Proposal_ID'].nunique().reset_index(name='Unique_Proposals_Count')

# Display the result
print(unique_proposals_count)

   Proposal_Type  Unique_Proposals_Count
0       approval                       2
1          basic                     114
2  ranked-choice                       7
3  single-choice                      19
4       weighted                       1


In [27]:
# Create a pie chart
fig = px.pie(unique_proposals_count, values='Unique_Proposals_Count', names='Proposal_Type', 
             title='Distribution of Proposals by Proposal Type',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_layout(template='plotly_dark')
# Show the chart
fig.show()
fig.write_html(r'Graphs\ditribution-proposals.html')

In [28]:
# Group the data by 'Proposal_Title' and sum the 'Voting_Power' for each proposal
proposal_voting_power = data.groupby('Proposal_Title')['Voting_Power'].sum().reset_index()

# Display the DataFrame
proposal_voting_power

,Proposal_Title,Voting_Power
0,Empowering Early Contributors: The community ...,1.227529e+08
1,0xGen STIP Proposal - Round 1,1.126770e+08
2,AIP 1.05: Return 700M $ARB to the DAO Treasury...,1.408547e+08
3,"AIP-1.1 - Lockup, Budget, Transparency",1.396229e+08
4,AIP-1.2 - Foundation and DAO Governance,1.387616e+08
...,...,...
138,[Non-Constitutional]: Arbitrum Stable Treasury...,1.327322e+08
139,dForce STIP Proposal - Round 1,1.395754e+08
140,iZUMi STIP Proposal - Round 1,1.548327e+08
141,tBTC STIP Proposal - Round 1,1.106967e+08


In [29]:
# Sort the DataFrame by 'Voting_Power' column in descending order
top_15_proposals = proposal_voting_power.sort_values(by='Voting_Power', ascending=False).head(15)

# Create the bar chart
fig = go.Figure(go.Bar(
    y=top_15_proposals['Proposal_Title'],  # Proposal titles on y-axis
    x=top_15_proposals['Voting_Power'],  # Voting power on x-axis
    orientation='h',  # Horizontal bar chart
    marker=dict(color=top_15_proposals['Voting_Power'],  # Color based on voting power
                colorscale='Viridis',  # Color scale
                colorbar=dict(title='Voting Power'))  # Color bar title
))

# Update layout
fig.update_layout(
    title='Top 30 Proposals by Total Voting Power',
    xaxis=dict(title='Total Voting Power'),
    yaxis=dict(title='Proposals'),
    showlegend=False
)

# Show the chart
fig.show()
fig.write_html(r'Graphs\top-15-power.html')